In [18]:
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
import datetime
from transformers import pipeline

/Users/thibaultrolland/anaconda3/envs/arcomenv/lib/python3.8/site-packages/huggingface_hub/utils/_runtime.py:184: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(


In [8]:
arcom_df = pd.read_csv("/Users/thibaultrolland/Projects/arcom/data/ARCOM.csv")

/var/folders/cf/xcc222mj0yg3dzy7mljbx99w0000gn/T/ipykernel_23177/777748067.py:1: DtypeWarning: Columns (37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  arcom_df = pd.read_csv("/Users/thibaultrolland/Projects/arcom/data/ARCOM.csv")


In [9]:
def clean_themes(themes):
    if type(themes) == str:
        themes = themes.split(',')
        themes = [theme.strip() for theme in themes]
        themes = [theme.lower() for theme in themes]
        return themes
    return []

arcom_df['clean_theme'] = arcom_df['Thème'].apply(clean_themes)
arcom_df['clean_date'] = [datetime.datetime.strptime(x, '%d/%m/%Y').date() for x in arcom_df['Date']]

In [10]:
most_frequent_themes = Counter([word for sublist in arcom_df['clean_theme'] for word in sublist])
most_frequent_themes.most_common()

[('pratique', 10319),
 ('interpellation', 10172),
 ('lien media', 9117),
 ('media internet', 8658),
 ('typographie', 8393),
 ('humour autre', 6660),
 ('efficacite', 6283),
 ('choix', 6017),
 ('gourmandise', 5795),
 ('nouveaute autre', 5713),
 ('effet visuel', 5689),
 ('illustration visuelle', 5673),
 ('joie', 5582),
 ('bonne humeur', 5498),
 ('personne celebre autre', 5389),
 ('enfant', 5361),
 ('signature sonore', 5290),
 ('anglais', 5173),
 ('interrogation', 5099),
 ('discours prix', 5086),
 ('gout', 4619),
 ('univers professionnel', 4616),
 ('service autre', 4381),
 ('musique celebre', 4308),
 ('comportement autre', 4277),
 ('couleur', 4142),
 ('animal autre', 4134),
 ('couple', 4098),
 ('musique autre', 3969),
 ('femme', 3962),
 ('quotidien', 3913),
 ('conseil', 3883),
 ('partenariat autre', 3807),
 ('famille', 3764),
 ('signe de qualite autre', 3511),
 ('innovation', 3473),
 ('complicite', 3458),
 ('etonnement', 3410),
 ('valorisation', 3400),
 ('univers urbain', 3390),
 ('signatu

In [13]:
f'number of distinct themes : {len(most_frequent_themes)}'

'number of distinct themes : 907'

In [63]:
themes_count = 20

top_themes = [word for word, count in most_frequent_themes.most_common(20)]

In [64]:
top_themes

['pratique',
 'interpellation',
 'lien media',
 'media internet',
 'typographie',
 'humour autre',
 'efficacite',
 'choix',
 'gourmandise',
 'nouveaute autre',
 'effet visuel',
 'illustration visuelle',
 'joie',
 'bonne humeur',
 'personne celebre autre',
 'enfant',
 'signature sonore',
 'anglais',
 'interrogation',
 'discours prix']

# Zero shot

In [77]:
zeroshot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0")

In [78]:
sample_len = 10

sample = arcom_df.sample(sample_len)
script_list = sample['Script'].tolist()

In [79]:

hypothesis_template = "Un des thèmes de ce script de publicité est : {}"
output_list = []

i = 0
for script in script_list:
    print(f'computing zero shot ad : {i+1} / {sample_len}', end='\r')
    output= zeroshot_classifier(script, top_themes, hypothesis_template=hypothesis_template, multi_label=True)
    output_list.append(output)
    i += 1

# adding true class to output
i = 0
for output in output_list:
    output['true_class'] = sample.iloc[i]['clean_theme']
    output['df_index'] = sample.index[i]
    i += 1

In [81]:
output_list

[{'sequence': 'Sophie Davant (1) : " Alors Claudette, vous me dites que vous êtes rassurée ...\n (1) ... depuis que votre fils a souscrit à VEILLER SUR MES PARENTS ...\n (1) ... avec votre accord, pourquoi ?\n Claudette (2) : " Mon fils n\'habite pas tout près et je n\'aime pas le déranger.\n L\'homme (3) : " Mais tu ne me dérange pas maman !\n (2) ... Oui, mais Astride, ma factrice passe deux fois par semaine. "\n (3) ... Et elle m\'envoie un compte rendu et avec la téléassistance 24 heures sur 24, ça me rassure. "\n (1) ... Alors avec la téléassistance couplée au facteur humain, ...\n (1) ... je comprends que vous soyez totalement serein. "\n Voix femme (4) : " Renseignez-vous à LA POSTE ou au 0 800 000 011. "',
  'labels': ['interrogation',
   'humour autre',
   'enfant',
   'lien media',
   'choix',
   'efficacite',
   'pratique',
   'signature sonore',
   'nouveaute autre',
   'discours prix',
   'joie',
   'typographie',
   'media internet',
   'illustration visuelle',
   'bonne 